In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp
import os as os
import pywt as py
import statistics as st
import random
import multiprocessing
from joblib import Parallel, delayed
import platform
from time import time as ti
from skimage.restoration import denoise_wavelet
from typing import List, Optional, Callable, Iterable
from itertools import islice
import pandas as pd
import dask.dataframe as dd

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '-', '-', '-']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')


#import CoreFunctions as cf
#from skimage.restoration import denoise_wavelet

In [2]:
HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
elif Computer == 'PortLap':
    location = 'C:\\users\\dhendrickson\\Desktop\\AccelData\\'

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/sciclone/scr10/dchendrickson01/Recordings2/'
    imageFolder = '/sciclone/scr10/dchendrickson01/Move3Dprint/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'Recordings2/'
elif Computer =='PortLap':
    rootfolder = location 
    folder = rootfolder + 'Recordings2\\'

In [3]:
Saving = False
location = folder
Titles = True
Ledgends = True
f = 0
freq = "2s"
prediction_length=50


In [4]:
files = ['230418 recording1.csv','230419 recording1.csv','230420 recording1.csv','230421 recording1.csv',
         '230418 recording2.csv','230419 recording2.csv','230420 recording2.csv','230421 recording2.csv']


In [5]:
def RollingStdDev(RawData, SmoothData, RollSize = 25):
    StdDevs = []
    for i in range(RollSize):
        Diffs = RawData[0:i+1]-SmoothData[0:i+1]
        Sqs = Diffs * Diffs
        Var = sum(Sqs) / (i+1)
        StdDev = np.sqrt(Var)
        StdDevs.append(StdDev)
    for i in range(len(RawData)-RollSize-1):
        j = i + RollSize
        Diffs = RawData[i:j]-SmoothData[i:j]
        Sqs = Diffs * Diffs
        Var = sum(Sqs) / RollSize
        StdDev = np.sqrt(Var)
        StdDevs.append(StdDev)  
    return StdDevs

def RollingSum(Data, Length = 100):
    RollSumStdDev = []
    for i in range(Length):
        RollSumStdDev.append(sum(Data[0:i+1]))
    for i in range(len(Data) - Length):
        RollSumStdDev.append(sum(Data[i:i+Length]))
    return RollSumStdDev

def SquelchPattern(DataSet, StallRange = 5000, SquelchLevel = 0.02):
    SquelchSignal = np.ones(len(DataSet))
    for i in range(len(DataSet)-2*StallRange):
        if np.average(DataSet[i:i+StallRange]) < SquelchLevel:
            SquelchSignal[i+StallRange]=0
    return SquelchSignal

def getVelocity(Acceleration, Timestamps = 0.003, Squelch = [], corrected = 0):
    velocity = np.zeros(len(Acceleration))
    Acceleration -= np.average(Acceleration)
    if len(Timestamps) == 1:
        dTime = np.ones(len(Acceleration),dtype=float) * Timestamps
    elif len(Timestamps) == len(Acceleration):
        dTime = np.zeros(len(Timestamps), dtype=float)
        dTime[0]=1
        for i in range(len(Timestamps)-1):
            j = i+1
            if Timestamps[j] > Timestamps[i]:
                dTime[j]=Timestamps[j]-Timestamps[i]
            else:
                dTime[j]=Timestamps[j]-Timestamps[i]+10000.0
        dTime /= 10000.0
    velocity[0] = Acceleration[0] * (dTime[0])
    for i in range(len(Acceleration)-1):
        j = i + 1
        if corrected ==2:
            if Squelch[j]==0:
                velocity[j]=0
            else:
                velocity[j] = velocity[i] + Acceleration[j] * dTime[j]                
        else:
            velocity[j] = velocity[i] + Acceleration[j] * dTime[j]
    if corrected == 1:
        PointVairance = velocity[-1:] / len(velocity)
        for i in range(len(velocity)):
            velocity[i] -=  PointVairance * i
    velocity *= 9.81
    return velocity

def MakeDTs(Seconds, Miliseconds):
    dts = np.zeros(len(Miliseconds), dtype=float)
    dts[0]=1
    for i in range(len(MiliSeconds)-1):
        j = i+1
        if Seconds[j]==Seconds[i]:
            dts[j]=Miliseconds[j]-Miliseconds[i]
        else:
            dts[j]=Miliseconds[j]-Miliseconds[i]+1000
    dts /= 10000
    return dts



In [6]:
for i in range(len(files)):
    files[i]=folder+files[i]


In [7]:
dataset = dd.read_csv(files, delimiter =", ", header=None)


/sciclone/home/dchendrickson01/.conda/envs/test/lib/python3.11/site-packages/dask/dataframe/io/csv.py:629: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  head = reader(BytesIO(b_sample), nrows=sample_rows, **head_kwargs)


In [8]:
dataset = dataset.rename(columns={0:"Day"})
dataset = dataset.rename(columns={1:"Second"})
dataset = dataset.rename(columns={2:"PartSec"})
dataset = dataset.rename(columns={3:"p"})
dataset = dataset.rename(columns={4:"h"})
dataset = dataset.rename(columns={5:"v"})
dataset = dataset.rename(columns={6:"Sensor"})

dataset['Day'] = dataset['Day'].apply(lambda x: str(x).zfill(6), meta=('Day', 'str'))
dataset['Second'] = dataset['Second'].apply(lambda x: str(x).zfill(6), meta=('Second', 'str'))
dataset['FracSec'] = dataset['PartSec'].apply(lambda x: str(x).zfill(4), meta=('PartSec', 'str'))

dataset['DayTime']=dataset.Day.str.cat([dataset.Second])
dataset['DayTime']=dataset.DayTime.str.cat([dataset.FracSec])

In [9]:
dataset['timestamp'] = dd.to_datetime(dataset.DayTime,unit='ns')

dataset["timestamps"] = dataset["timestamp"]

In [10]:
dataset.head


<bound method _Frame.head of Dask DataFrame Structure:
                    Day  Second PartSec        p        h        v Sensor FracSec DayTime       timestamp      timestamps
npartitions=100                                                                                                          
                 object  object   int64  float64  float64  float64  int64  object  object  datetime64[ns]  datetime64[ns]
                    ...     ...     ...      ...      ...      ...    ...     ...     ...             ...             ...
...                 ...     ...     ...      ...      ...      ...    ...     ...     ...             ...             ...
                    ...     ...     ...      ...      ...      ...    ...     ...     ...             ...             ...
                    ...     ...     ...      ...      ...      ...    ...     ...     ...             ...             ...
Dask Name: assign, 30 graph layers>

Works to here

In [11]:
asdf

NameError: name 'asdf' is not defined

In [12]:
dataset["p"] = dataset.p - np.average(dataset.p)
dataset["h"] = dataset.h - np.average(dataset.h)
dataset["v"] = dataset.v - np.average(dataset.v)
dataset["r"] = np.sqrt(dataset.p**2 + dataset.h**2 + dataset.v**2)


dataset["smoothP"] = denoise_wavelet(dataset.p, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
dataset["SmoothH"] = denoise_wavelet(dataset.h, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')
dataset["SmoothV"] = denoise_wavelet(dataset.v, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')

StdDevsV = RollingStdDev(dataset.v, dataset.SmoothV)
StdDevsV.append(0)
StdDevsV = np.asarray(StdDevsV)
SmoothDevV = denoise_wavelet(StdDevsV, method='VisuShrink', mode='soft', wavelet_levels=3, wavelet='sym2', rescale_sigma='True')

Max = np.max(SmoothDevV)
buckets = int(Max / 0.005) + 1
bins = np.linspace(0,buckets*0.005,buckets+1)
counts, bins = np.histogram(SmoothDevV,bins=bins)

CummCount = 0
HalfWay = 0
for i in range(len(counts)):
    CummCount += counts[i]
    if CummCount / len(SmoothDevZ) >= 0.5:
        if HalfWay == 0:
            HalfWay = i

SquelchLevel = bins[HalfWay] 
dataset["IsMoving"] = SquelchPattern(SmoothDevV, 4000, SquelchLevel)

dataset["velocity"] = getVelocity(dataset.r, dataset.PartSec, dataset.IsMoving, 2)

/sciclone/home/dchendrickson01/.conda/envs/test/lib/python3.11/site-packages/dask/dataframe/io/csv.py:195: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = reader(bio, **kwargs)


OutOfBoundsDatetime: Parsing "2304190000001825" to datetime overflows, at position 0

In [ ]:
asdf

In [ ]:
data = Parallel(n_jobs=8)(delayed(MakeDataframe)(file) for file in files)
dataset = pd.concat(data)[["timestamps","x","y","z","r","IsMoving","Velocity"]]
dataset.head()

In [ ]:
from gluonts.torch import DeepAREstimator
#from gluonts.torch.trainer import Trainer
from gluonts.evaluation import make_evaluation_predictions, Evaluator


def train_and_predict(dataset, estimator):
    predictor = estimator.train(dataset)
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset, predictor=predictor
    )
    evaluator = Evaluator(quantiles=(np.arange(20) / 20.0)[1:])
    agg_metrics, item_metrics = evaluator(ts_it, forecast_it, num_series=len(dataset))
    return agg_metrics["MSE"]

In [ ]:
estimator = DeepAREstimator(
    freq=freq, prediction_length=prediction_length
)

In [ ]:
from gluonts.dataset.pandas import PandasDataset

In [ ]:
#ds = PandasDataset(dataset, target="x", freq=freq)
ds = PandasDataset.from_long_dataframe(
    dataframe=dataset,
    item_id="timestamp",
    timestamp="timestamps",
    freq="ns",
)

In [ ]:
train_and_predict(dataset, estimator)